# High Performance Python Notes
> My notes for the book "High Performance Python"
- toc: true 
- badges: false
- comments: true
- categories: [programming]
<!-- - image: images/normal-dist.jpg -->

## Profiling to find bottlenecks

Let's say we want to calculate how much time a function `foo` in our program takes to run. We can do this using `time` module and calculating the time taken wherever the function is called

In [ ]:
import time

start = time.time() ## Noting the start time

foo(*args, **kwargs) ## calling the function

end = time.time() ## noting the end time

print(f'The function {foo.__name__} took {end-start} seconds to run')

This approach however requires us to write the code for calculating the time taken by function everywhere the function is called. If the function is called numerous times, this approach can clutter our program.

A better approach would be to use decorator

#### Using Decorators

In [2]:
def timer_func(func):
    
    def time_measurer(*args, **kwargs):
        start = time.time()
        
        reult = func(*args, **kwargs)
        
        end = time.time()
        
        print(f'The function {func.__name__} took {end-start} seconds to run')
        
        return result
    
    return time_measurer

Now we only need to "decorate"  the function as follows

In [5]:
@timer_func
def foo(*args, **kwargs):
    ...
    
    

The above code snippet is just a fancy way of saying `foo = timer_func(foo)`.  With this approach, we only need to write the code for calculating the time taken once and then using a decorator we can convert `foo` into a function that prints the time taken and returns the result. Moreover, we can time any function using this decorator

But there's one problem with this approach. Whenever the function `foo` will be called it will print out `The function time_measurer took 10 seconds to run`. This is because `timer_func` returns a function named "time_measurer". We can circumvent this issue by a small fix.

In [6]:
from functools import wraps

def timer_func(func):
    
    @wraps(func)
    def time_measurer(*args, **kwargs):
        start = time.time()
        
        reult = func(*args, **kwargs)
        
        end = time.time()
        
        print(f'The function {func.__name__} took {end-start} seconds to run')
        
        return result
    
    return time_measurer

`wraps` decorator forces the function `time_measurer` to have the same attributes as that of `func`.

#### Using magic commands

In Jupyter notebooks we can use magic `%timeit` for timing the function. This will return mean and standard deviation of run time of several calls to the function

In [11]:
import julia_set

%timeit julia_set.calc_pure_python(desired_width = 1000, max_iterations = 300)

6.73 s ± 112 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


`timeit` can also be run from command line as:

In [9]:
! python -m timeit -n 5 -r 2 -s "import julia_set" "julia_set.calc_pure_python(desired_width = 1000, max_iterations = 300)"

5 loops, best of 2: 6.96 sec per loop


> Note: Running `timeit` using magic command return mean of all the runs while while running it from command line displays the time of the best run